In [1]:
from nnsight import LanguageModel

/root/reward_hacking_sae_dashboard/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from nnterp import StandardizedTransformer

model = StandardizedTransformer("openai/gpt-oss-20b", device_map="cpu", dispatch=True, dtype='bfloat16')

/root/reward_hacking_sae_dashboard/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16
Loading checkpoint shards: 100%|██████████| 3/3 [00:24<00:00,  8.01s/it]
2025-10-22 20:24:31.821 | WARNING  | nnterp.utils:try_with_scan:141 - Error when trying to scan the model - using .trace() instead (which will dispatch the model)...
2025-10-22 20:24:42.348 | WARNING  | nnterp.utils:try_with_scan:152 - Using trace() succeed! Error when trying to scan the model:


Traceback (most recent call last):
  File "/root/reward_hacking_sae_dashboard/.venv/lib/python3.12/site-packages/nnsight/intervention/backends/execution.py", line 21, in __call__
    tr

In [16]:
import torch

In [7]:
from nnterp.utils import display_source

In [22]:
output

tensor([[24912,  2375,   672]])

In [23]:
tokenizer.decode(output[0], skip_special_tokens=False)

'hello world",'

In [40]:
with torch.no_grad():
    with model.generate("and the east is actually ", max_new_tokens=1) as tracer:
        shape = model.model.layers[0].mlp.router.source.F_linear_0.output.shape
        #model.model.layers[0].mlp.router.source.F_linear_0.output.value = torch.randn(shape)
        output = model.generator.output.save()
    # Decode
decoded = tokenizer.decode(output[0], skip_special_tokens=False)

In [36]:
linear.shape

torch.Size([2, 32])

In [39]:
decoded

'and the east is actually 170'

In [11]:
model.model.layers[0].mlp

GptOssMLP(
  (router): GptOssTopKRouter()
  (experts): GptOssExperts()
)

In [27]:
display_source(model.model.layers[0].mlp.router.source.F_linear_0)

```py
model.model.layers.0.mlp.router.F_linear_0:

    def forward(self, hidden_states):
        hidden_states = hidden_states.reshape(-1, self.hidden_dim)
    --> router_logits = F.linear(hidden_states, self.weight, self.bias)  # (seq_len, num_experts) <--
        router_top_value, router_indices = torch.topk(router_logits, self.top_k, dim=-1)  # (seq_len, top_k)
        router_top_value = torch.nn.functional.softmax(router_top_value, dim=1, dtype=router_top_value.dtype)
        router_scores = torch.zeros_like(router_logits).scatter_(1, router_indices, router_top_value)
        return router_scores, router_indices
```

In [14]:
display_source(model.model.layers[0].mlp.experts.source)

```py
                                   * def forward(self, hidden_states: torch.Tensor, router_indices=None, routing_weights=None) -> torch.Tensor:
                                   0     """
                                   1     When training it is more efficient to just loop over the experts and compute the output for each expert
                                   2     as otherwise the memory would explode.
                                   3 
                                   4     For inference we can sacrifice some memory and compute the output for all experts at once. By repeating the inputs.
                                   5 
                                   6     Args:
                                   7         hidden_states (torch.Tensor): (batch_size, seq_len, hidden_size)
                                   8         selected_experts (torch.Tensor): (batch_size * token_num, top_k)
                                   9         routing_weights (torch.Tensor): (batch_size * token_num, num_experts)
                                  10     Returns:
                                  11         torch.Tensor
                                  12     """
                                  13     batch_size = hidden_states.shape[0]
 hidden_states_reshape_0       -> 14     hidden_states = hidden_states.reshape(-1, self.hidden_size)  # (num_tokens, hidden_size)
                                  15     num_experts = routing_weights.shape[1]
                                  16     if hidden_states.device.type == "cpu" or self.training:
 torch_zeros_like_0            -> 17         next_states = torch.zeros_like(hidden_states, dtype=hidden_states.dtype, device=hidden_states.device)
 torch_no_grad_0               -> 18         with torch.no_grad():
 torch_nn_functional_one_hot_0 -> 19             expert_mask = torch.nn.functional.one_hot(
                                  20                 router_indices, num_classes=num_experts + 1
                                  21             )  # masking is also a class
 expert_mask_permute_0         -> 22             expert_mask = expert_mask.permute(2, 1, 0)
                                  23             # we sum on the top_k and on the sequence length to get which experts
                                  24             # are hit this time around
 expert_mask_sum_0             -> 25             expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero()
 torch_greater_0               ->  +             ...
 nonzero_0                     ->  +             ...
                                  26         for expert_idx in expert_hit[:]:
                                  27             # expert_idx only have 1 element, so we can use scale for fast indexing
                                  28             expert_idx = expert_idx[0]
                                  29             # skip masking index
                                  30             if expert_idx == num_experts:
                                  31                 continue
 torch_no_grad_1               -> 32             with torch.no_grad():
 torch_where_0                 -> 33                 _, token_idx = torch.where(expert_mask[expert_idx])
                                  34             current_state = hidden_states[token_idx]
                                  35             gate_up = current_state @ self.gate_up_proj[expert_idx] + self.gate_up_proj_bias[expert_idx]
                                  36             gate, up = gate_up[..., ::2], gate_up[..., 1::2]
 gate_clamp_0                  -> 37             gate = gate.clamp(min=None, max=self.limit)
 up_clamp_0                    -> 38             up = up.clamp(min=-self.limit, max=self.limit)
 torch_sigmoid_0               -> 39             glu = gate * torch.sigmoid(gate * self.alpha)
                                  40             gated_output = (up + 1) * glu
                                  41             out = gated_output @ self.down_proj[expert_idx] + self.down_proj_bias[expert_idx]
                                  42             weighted_output = out * routing_weights[token_idx, expert_idx, None]
 weighted_output_to_0          -> 43             next_states.index_add_(0, token_idx, weighted_output.to(hidden_states.dtype))
 next_states_index_add__0      ->  +             ...
 next_states_view_0            -> 44         next_states = next_states.view(batch_size, -1, self.hidden_size)
                                  45     else:
 hidden_states_repeat_0        -> 46         hidden_states = hidden_states.repeat(num_experts, 1)
 hidden_states_view_0          -> 47         hidden_states = hidden_states.view(num_experts, -1, self.hidden_size)
 torch_bmm_0                   -> 48         gate_up = torch.bmm(hidden_states, self.gate_up_proj) + self.gate_up_proj_bias[..., None, :]
                                  49         gate, up = gate_up[..., ::2], gate_up[..., 1::2]
 gate_clamp_1                  -> 50         gate = gate.clamp(min=None, max=self.limit)
 up_clamp_1                    -> 51         up = up.clamp(min=-self.limit, max=self.limit)
 torch_sigmoid_1               -> 52         glu = gate * torch.sigmoid(gate * self.alpha)
 torch_bmm_1                   -> 53         next_states = torch.bmm(((up + 1) * glu), self.down_proj)
                                  54         next_states = next_states + self.down_proj_bias[..., None, :]
 next_states_view_1            -> 55         next_states = next_states.view(num_experts, batch_size, -1, self.hidden_size)
 routing_weights_transpose_0   -> 56         next_states = next_states * routing_weights.transpose(0, 1).view(num_experts, batch_size, -1)[..., None]
 view_0                        ->  +         ...
 next_states_sum_0             -> 57         next_states = next_states.sum(dim=0)
                                  58     return next_states
                                  59 
```